In [12]:
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

# Display options
pd.options.display.max_columns = None
%matplotlib inline

In [13]:
con = pyodbc.connect('Trusted_Connection=yes', driver='{SQL Server}', server='dev-sql05', database='Delphi')

cur = con.cursor()

querystring = 'SELECT * FROM mdl_AcuteInpatient1'

df_AcuteInpatient = pd.read_sql(querystring,con)

In [17]:
df_AcuteInpatient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140297 entries, 0 to 140296
Data columns (total 57 columns):
RowID                         140297 non-null int64
PatientID                     140297 non-null int64
DateMonthlyID                 140297 non-null int64
IsAsthma                      140297 non-null int64
IsCOPD                        140297 non-null int64
IsDepressionOrBH              140297 non-null int64
IsDiabetes                    140297 non-null int64
IsHeartFailure                140297 non-null int64
IsHypertension                140297 non-null int64
IsOsteoporosis                140297 non-null int64
IsHeartAttack                 140297 non-null int64
IsAlzheimersOrRelated         140297 non-null int64
IsChronicKidney               140297 non-null int64
CurrentMonthOPPCnt            140297 non-null int64
OPPLast6Mths                  140297 non-null int64
OPPMonthlyAvg                 140297 non-null float64
HasPreviousAdmit              140297 non-null int64
Has

In [18]:
# Convert all possible columns to int, ignore if not int.
df_AcuteInpatient = df_AcuteInpatient.apply(pd.to_numeric, errors='ignore')

In [14]:
df_AcuteInpatient.head()

,RowID,PatientID,DateMonthlyID,IsAsthma,IsCOPD,IsDepressionOrBH,IsDiabetes,IsHeartFailure,IsHypertension,IsOsteoporosis,IsHeartAttack,IsAlzheimersOrRelated,IsChronicKidney,CurrentMonthOPPCnt,OPPLast6Mths,OPPMonthlyAvg,HasPreviousAdmit,Has1MonthSurgery,Has3MonthSurgery,IsObeseClass1,IsObeseClass2,IsObeseClass3,IsObese,Gender,Race,Ethnicity,PreferredLanguageDesc,ZipCode,City,UsualProviderTitle,MaritalStatus,IsActivelyManaged,IsPrimaryCare,IsSpecialtyCare,IsPHPPatient,HasMedicareIns,HasMedicaidIns,HasLiabilityIns,HasTricareIns,HasCollection,HasWellcareIns,HasCommunityHealthIns,HasHealthNetIns,DateOfBirth,Age,Over21AgeBracketID,CntInpatient,CntIsAcuteInpatient,CntIsSubAcuteInpatient,CntIsERClaim,IsAcuteER,CntInpatientPrev,CntIsAcuteInpatientPrev,CntIsSubAcuteInpatientPrev,CntIsERClaimPrev,IsAcuteERPrev,HasAcuteERIn6Mths
0,2134,775,327,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0,0,0,1,0,0,1,F,White,Non-Hispanic or Latino,English,06791,Harwinton,MD,M,1,1,0,1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1983-05-23,34,2,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0
1,2145,821,326,0,0,1,0,0,1,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,F,White,Non-Hispanic or Latino,English,06759,Litchfield,MD,M,1,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1963-03-18,54,4,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0
2,2167,877,322,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,1,0,0,1,M,White,Non-Hispanic or Latino,English,06787,Thomaston,MD,M,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1941-09-11,75,5,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
3,2178,820,327,0,0,1,1,0,1,0,0,1,0,0,0,0.0,0,0,0,0,0,1,1,F,White,Non-Hispanic or Latino,English,06479,Plantsville,MD,S,1,1,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1937-03-15,80,5,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
4,2189,838,318,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,M,White,Non-Hispanic or Latino,English,06032,Farmington,MD,None,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1953-03-13,63,4,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0


In [21]:
# Create sets of Target and Nontarget for easier analysis.

df_Target = df_AcuteInpatient[df_AcuteInpatient.HasAcuteERIn6Mths == 1]

df_NonTarget = df_AcuteInpatient[df_AcuteInpatient.HasAcuteERIn6Mths == 0]

In [20]:
df_Target.describe()

,RowID,PatientID,DateMonthlyID,IsAsthma,IsCOPD,IsDepressionOrBH,IsDiabetes,IsHeartFailure,IsHypertension,IsOsteoporosis,IsHeartAttack,IsAlzheimersOrRelated,IsChronicKidney,CurrentMonthOPPCnt,OPPLast6Mths,OPPMonthlyAvg,HasPreviousAdmit,Has1MonthSurgery,Has3MonthSurgery,IsObeseClass1,IsObeseClass2,IsObeseClass3,IsObese,ZipCode,IsActivelyManaged,IsPrimaryCare,IsSpecialtyCare,IsPHPPatient,HasMedicareIns,HasMedicaidIns,HasLiabilityIns,HasTricareIns,HasCollection,HasWellcareIns,HasCommunityHealthIns,HasHealthNetIns,DateOfBirth,Age,Over21AgeBracketID,CntInpatient,CntIsAcuteInpatient,CntIsSubAcuteInpatient,CntIsERClaim,IsAcuteER,CntInpatientPrev,CntIsAcuteInpatientPrev,CntIsSubAcuteInpatientPrev,CntIsERClaimPrev,IsAcuteERPrev,HasAcuteERIn6Mths
count,2.758000e+03,2758.000000,2758.000000,2758.000000,2758.000000,2758.000000,2758.000000,2758.000000,2758.000000,2758.000000,2758.000000,2758.000000,2758.000000,2758.000000,2758.000000,2758.000000,2758.0,2758.000000,2758.000000,2758.000000,2758.000000,2758.000000,2758.000000,2.758000e+03,2758.000000,2758.000000,2758.000000,2758.000000,2151.000000,2151.000000,2151.000000,2151.000000,2151.000000,2151.0,2151.000000,2151.000000,2.758000e+03,2758.000000,2758.000000,2758.000000,2758.000000,2758.000000,2758.000000,2758.000000,2495.000000,2495.000000,2495.000000,2495.000000,2495.000000,2758.0
mean,1.157362e+06,434977.680203,319.648658,0.076142,0.132705,0.285714,0.192893,0.092458,0.485497,0.076142,0.010152,0.080131,0.213198,7.996737,22.418782,6.459349,1.0,0.407904,0.559463,0.168963,0.074692,0.067440,0.301305,4.976415e+05,0.150109,0.072516,0.010877,0.113851,0.720595,0.406788,0.023245,0.000930,0.202696,0.0,0.005579,0.081358,-8.933172e+17,74.994924,4.724801,0.533720,0.368020,0.165700,1.756345,0.186004,0.610020,0.379158,0.230862,1.820040,0.183166,1.0
std,6.285895e+05,237364.897836,4.673781,0.265273,0.339317,0.451836,0.394642,0.289724,0.499880,0.265273,0.100264,0.271544,0.409641,16.241054,60.871447,13.297793,0.0,0.491534,0.496542,0.374787,0.262941,0.250828,0.458908,5.611409e+06,0.357243,0.259388,0.103745,0.317687,0.448811,0.491349,0.150716,0.030486,0.402102,0.0,0.074500,0.273447,4.374532e+17,13.837388,0.703925,0.964432,0.701172,0.470098,3.490609,0.389181,0.984554,0.689978,0.553122,3.535715,0.386880,0.0
min,1.367000e+03,712.000000,313.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.034000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-1.744243e+18,23.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,6.086000e+05,228526.000000,316.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.066000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-1.218456e+18,68.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,1.193270e+06,441871.000000,319.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.429000,1.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,6.357000e+03,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-9.579600e+17,77.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
75%,1.740027e+06,657378.750000,323.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,9.000000,19.000000,7.400000,1.0,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,6.489000e+03,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-6.651720e+17,85.000000,5.0

In [22]:
df_NonTarget.describe()

,RowID,PatientID,DateMonthlyID,IsAsthma,IsCOPD,IsDepressionOrBH,IsDiabetes,IsHeartFailure,IsHypertension,IsOsteoporosis,IsHeartAttack,IsAlzheimersOrRelated,IsChronicKidney,CurrentMonthOPPCnt,OPPLast6Mths,OPPMonthlyAvg,HasPreviousAdmit,Has1MonthSurgery,Has3MonthSurgery,IsObeseClass1,IsObeseClass2,IsObeseClass3,IsObese,ZipCode,IsActivelyManaged,IsPrimaryCare,IsSpecialtyCare,IsPHPPatient,HasMedicareIns,HasMedicaidIns,HasLiabilityIns,HasTricareIns,HasCollection,HasWellcareIns,HasCommunityHealthIns,HasHealthNetIns,DateOfBirth,Age,Over21AgeBracketID,CntInpatient,CntIsAcuteInpatient,CntIsSubAcuteInpatient,CntIsERClaim,IsAcuteER,CntInpatientPrev,CntIsAcuteInpatientPrev,CntIsSubAcuteInpatientPrev,CntIsERClaimPrev,IsAcuteERPrev,HasAcuteERIn6Mths
count,1.375390e+05,137539.000000,137539.000000,137539.000000,137539.000000,137539.000000,137539.000000,137539.000000,137539.000000,137539.000000,137539.000000,137539.000000,137539.000000,137539.000000,137539.000000,137539.00000,137539.000000,137539.000000,137539.000000,137539.000000,137539.000000,137539.000000,137539.000000,1.375390e+05,137539.000000,137539.000000,137539.000000,137539.000000,68798.000000,68798.000000,68798.000000,68798.000000,68798.000000,68798.000000,68798.000000,68798.000000,1.375390e+05,137539.000000,137539.000000,137539.000000,137539.000000,137539.000000,137539.000000,137539.0,93564.000000,93564.000000,93564.000000,93564.000000,93564.0,137539.0
mean,1.048751e+06,393717.921033,321.074612,0.091109,0.051753,0.285715,0.117574,0.020031,0.403413,0.052872,0.004188,0.028050,0.099172,1.428468,3.742822,0.84113,0.002683,0.177419,0.303688,0.199289,0.089262,0.061139,0.347327,4.428293e+05,0.330539,0.140956,0.015734,0.180225,0.539681,0.233728,0.063200,0.007704,0.366290,0.000349,0.009710,0.063156,-4.412839e+17,60.775453,3.961284,0.029228,0.016352,0.012840,0.038345,0.0,0.037953,0.020681,0.017218,0.041768,0.0,0.0
std,6.051562e+05,227591.607180,4.864404,0.287765,0.221528,0.451756,0.322104,0.140106,0.490584,0.223779,0.064579,0.165117,0.298894,6.685251,21.523285,4.02387,0.051727,0.382024,0.459851,0.399467,0.285123,0.239586,0.476122,9.869789e+06,0.470409,0.347978,0.124444,0.384376,0.498427,0.423204,0.243323,0.087433,0.481793,0.018674,0.098058,0.243245,5.522736e+17,17.493545,1.197277,0.224120,0.145513,0.129578,0.382624,0.0,0.255216,0.164212,0.147850,0.411078,0.0,0.0
min,1.100000e+01,5.000000,313.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.780000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.051222e+18,22.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
25%,5.242545e+05,196820.500000,317.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.067000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-8.452512e+17,49.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
50%,1.048905e+06,392084.000000,321.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.370000e+03,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.621536e+17,61.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
75%,1.571102e+06,588787.000000,325.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,6.480000e+03,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0

In [24]:
df_AcuteInpatient[['Gender','HasAcuteERIn6Mths']].groupby(['Gender'], as_index=False).mean().sort_values(by='HasAcuteERIn6Mths', ascending=False)

,Gender,HasAcuteERIn6Mths
1,M,0.021186
0,F,0.018557


In [25]:
df_AcuteInpatient[['Race','HasAcuteERIn6Mths']].groupby(['Race'], as_index=False).mean().sort_values(by='HasAcuteERIn6Mths', ascending=False)

,Race,HasAcuteERIn6Mths
4,Native Hawaiian or Other Pacific Islander,0.038462
5,Not Reported,0.029575
3,Declined,0.026886
2,Black or African American,0.020242
6,Unknown,0.018419
7,White,0.017078
0,American Indian or Alaska Native,0.006410
1,Asian,0.002527


In [28]:
df_AcuteInpatient[['Ethnicity','HasAcuteERIn6Mths']].groupby(['Ethnicity'], as_index=False).mean().sort_values(by='HasAcuteERIn6Mths', ascending=False)

,Ethnicity,HasAcuteERIn6Mths
4,Unknown,0.022639
3,Not Reported,0.022395
0,Declined,0.020936
2,Non-Hispanic or Latino,0.016470
1,Hispanic or Latino,0.012748


In [29]:
df_AcuteInpatient[['PreferredLanguageDesc','HasAcuteERIn6Mths']].groupby(['PreferredLanguageDesc'], as_index=False).mean().sort_values(by='HasAcuteERIn6Mths', ascending=False)

,PreferredLanguageDesc,HasAcuteERIn6Mths
30,Thai (Laotian),0.307692
8,English,0.017595
29,Spanish,0.015827
24,Polish,0.000000
19,Korean,0.000000
20,Mandarin,0.000000
21,Mon-Khmer (Cambodian),0.000000
22,Navaho,0.000000
23,Persian,0.000000
0,Amharic,0.000000


In [31]:
df_AcuteInpatient[['City','HasAcuteERIn6Mths']].groupby(['City'], as_index=False).mean().sort_values(by='HasAcuteERIn6Mths', ascending=False)

,City,HasAcuteERIn6Mths
294,Glen Cove,1.000000
40,Ballston Spa,1.000000
768,Shirley,1.000000
441,Little River,0.833333
397,Key West,0.833333
851,Templeton,0.750000
612,Olney,0.750000
513,Montville,0.545455
704,Rochester,0.500000
191,Delray Beach,0.454545


In [35]:
df_AcuteInpatient[['UsualProviderTitle','HasAcuteERIn6Mths']].groupby(['UsualProviderTitle'], as_index=False).mean().sort_values(by='HasAcuteERIn6Mths', ascending=False)

,UsualProviderTitle,HasAcuteERIn6Mths
0,APRN,0.049470
3,MD,0.020188
2,DO,0.012562
4,PA-C,0.004717
1,AuD,0.000000
5,PT,0.000000


In [36]:
df_AcuteInpatient[['MaritalStatus','HasAcuteERIn6Mths']].groupby(['MaritalStatus'], as_index=False).mean().sort_values(by='HasAcuteERIn6Mths', ascending=False)

,MaritalStatus,HasAcuteERIn6Mths
5,W,0.041603
1,D,0.019554
3,S,0.019100
6,X,0.017316
2,M,0.014184
0,,0.000000
4,U,0.000000


In [38]:
df_AcuteInpatient[['Over21AgeBracketID','HasAcuteERIn6Mths']].groupby(['Over21AgeBracketID'], as_index=False).mean().sort_values(by='HasAcuteERIn6Mths', ascending=False)

,Over21AgeBracketID,HasAcuteERIn6Mths
4,5,0.036654
3,4,0.007599
2,3,0.005781
0,1,0.003712
1,2,0.003529


In [39]:
df_AcuteInpatient[['CntInpatient','HasAcuteERIn6Mths']].groupby(['CntInpatient'], as_index=False).mean().sort_values(by='HasAcuteERIn6Mths', ascending=False)

,CntInpatient,HasAcuteERIn6Mths
8,8,1.000000
9,9,1.000000
6,6,0.750000
7,7,0.750000
5,5,0.416667
4,4,0.383562
3,3,0.377880
2,2,0.279558
1,1,0.196863
0,0,0.013734


In [40]:
df_AcuteInpatient[['CntIsAcuteInpatient','HasAcuteERIn6Mths']].groupby(['CntIsAcuteInpatient'], as_index=False).mean().sort_values(by='HasAcuteERIn6Mths', ascending=False)

,CntIsAcuteInpatient,HasAcuteERIn6Mths
7,7,1.000000
8,8,1.000000
9,9,1.000000
5,5,0.750000
6,6,0.500000
3,3,0.444444
4,4,0.400000
2,2,0.396667
1,1,0.266555
0,0,0.014295


In [41]:
df_AcuteInpatient[['CntIsSubAcuteInpatient','HasAcuteERIn6Mths']].groupby(['CntIsSubAcuteInpatient'], as_index=False).mean().sort_values(by='HasAcuteERIn6Mths', ascending=False)

,CntIsSubAcuteInpatient,HasAcuteERIn6Mths
4,4,0.400000
3,3,0.363636
2,2,0.277580
1,1,0.171885
0,0,0.017346
5,5,0.000000


In [42]:
df_AcuteInpatient[['CntIsERClaim','HasAcuteERIn6Mths']].groupby(['CntIsERClaim'], as_index=False).mean().sort_values(by='HasAcuteERIn6Mths', ascending=False)

,CntIsERClaim,HasAcuteERIn6Mths
29,51,1.000000
21,21,1.000000
18,18,1.000000
16,16,1.000000
22,22,1.000000
24,25,1.000000
19,19,1.000000
25,26,1.000000
26,27,1.000000
27,35,1.000000


In [43]:
df_AcuteInpatient[['IsAcuteERPrev','HasAcuteERIn6Mths']].groupby(['IsAcuteERPrev'], as_index=False).mean().sort_values(by='HasAcuteERIn6Mths', ascending=False)

,IsAcuteERPrev,HasAcuteERIn6Mths
1,1.0,1.000000
0,0.0,0.021318
